# Federated Learning

Federated learning is a machine learning paradigm that enables decentralized training of a shared model by multiple clients while preserving data privacy. The main idea behind this new paradigm is that each client trains a local model on its own data and then sends only the model updates to a central server, rather than sending the raw data. This allows the model to be trained on a large amount of data without compromising data privacy.

Federated learning was first proposed by Google in 2016 (McMahan et al., 2016) and has since been applied in various fields, such as healthcare (Hard et al., 2018), finance (Yoon et al., 2018), and natural language processing (Li et al., 2020).  For example, federated learning could be used to train a model that can make personalized recommendations for each user without requiring the raw data from each user to be shared with a central server. This is the mechanism by which the model is trained on data, while adhering to data privacy requirements.

Federated learning is a machine learning approach that offers numerous advantages over traditional centralized methods. Firstly, by leveraging distributed data stores, federated learning can scale to handle significantly larger datasets. Secondly, it prioritizes data privacy by avoiding the transmission of raw data to a central server. Finally, federated learning enables collaboration among multiple clients, allowing them to jointly train a shared model without compromising the security of their individual data. Overall, these benefits make federated learning a promising approach for machine learning in fields where data privacy is of the utmost importance.

Federated learning is a process in which a central server distributes a machine learning model to multiple devices. Each device trains the model on its local data and sends the updated model back to the central server. The central server then aggregates the updates from each device to improve the global model. This process is repeated until the model converges and can generate accurate predictions on new data. The key concepts within this process are:


* Client: refers to a device or edge node that holds a local dataset and actively participates in the training of the federated model.
* Server: represents the central entity that coordinates the training of the federated model and receives model updates from the clients to aggregate into a new version of the global model.
* Federated dataset: the collection of decentralized datasets from different clients that are used to train the federated model through collaborative learning.
* Federated model: a machine learning model that is trained on the federated dataset using federated learning to make accurate predictions on new data while preserving the privacy of each client's data.
* Federated optimization: refers to the process of training the federated model using the decentralized data and model updates from the clients, which enables the model to generalize better on unseen data while preserving the privacy of the clients.
* Aggregation: the process of combining the model updates received from the clients into a new version of the global model. This can be done using various methods such as weighted averaging or other approaches.
* Rounds: refer to the number of times a federated model is distributed among clients after performing an aggregation to train the model further. The process is repeated until the model converges and achieves a satisfactory level of accuracy.


Federated learning is a relatively new approach, and as such, there are few libraries available that have adapted to it. The main actors in this space are TensorFlow Federated, PySyft, OpenMined, and Flower. Of these, TensorFlow Federated is a notable mention, although it is currently only a theoretical approach, as it does not allow for the deployment of the solution and only simulates the federated space. In contrast, Flower allows for the distribution of federated learning, although the necessary modifications can be somewhat challenging. For this tutorial, we have chosen Flower due to its more user-friendly approach and potential for future use.



# Introduction to Flower (FLWR)

Flower is a Python library that offers tools for implementing the communication and coordination aspects of federated learning. Its design emphasizes ease of use and scalability. It's important to note that Flower is not a learning framework in itself, and as such, it wraps other machine learning frameworks like TensorFlow, PyTorch, or Scikit-learn in the communication layer to enable federated learning.

To use Flower for federated learning, you will need to install the library:


In [1]:
try:
    import flwr as fl
except ImportError as err:
    !pip install flwr[simulation]
    
try:
    import tensorflow as tf
except ImportError as err:
    !pip install tensorflow

2024-03-14 10:28:12.354843: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-14 10:28:12.355273: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-14 10:28:12.358097: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-14 10:28:12.393680: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 10:28:13.018764: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

When setting up a simulation environment, it's best to use the *simulation* keyword with the command to ensure the appropriate environment is loaded. On the other hand, if you plan to use Flower in a distributed setup, the command should be `!pip install flwr` on both the server and client devices. After installing the `flwr` package, you can import it into your Python code using the following statement:

In [2]:
import flwr as fl
import tensorflow as tf

FLWR provides a range of classes and functions that you can use to set up a federated learning environment, train and evaluate a model, and implement regular updates to the model. You can refer to the FLWR [documentation](https://flower.dev/docs/quickstart-tensorflow.html)  for more information. Before proceeding, it's important to note that the model you define must be serializable so that it can be sent through the network. Not all models are suitable for federated learning. For this example, we'll be using an Artificial Neural Network (ANN) based on TensorFlow, specifically Keras.

In [3]:
# Define a simple model using TensorFlow
def generate_ann():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

As we will be using a Deep Learning model defined in TensorFlow, it's recommended to load the data into a `Dataset` class to enable the framework to leverage any available hardware acceleration (such as a GPU on the nodes). However, due to some limitations of the framework in order to serialize the data, it has to be done manualy with the following lines of code

In [4]:
import numpy as np
import tensorflow as tf

NUM_CLIENTS = 5

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def split_index(a, n):
    s = np.array_split(np.arange(len(a)), n)
    return s


# Code to load the dataset
def load_datasets(num_clients: int):
    # Distribute it to train and test set
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    # Normalize data
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    x_train, y_train = x_train[:10000], y_train[:10000]
    x_test, y_test = x_test[:1000], y_test[:1000]

    # Randomize the datasets
    x_train, y_train = unison_shuffled_copies(x_train, y_train)
    x_test, y_test = unison_shuffled_copies(x_test, y_test)

    # Split training set into NUM_CLIENTS partitions to simulate the individual dataset
    train_index = split_index(x_train, num_clients)
    test_index = split_index(x_test, num_clients)

    # Split each partition
    train_ds = []
    val_ds = []
    test_ds = []
    for cid in range(num_clients):
        val_size = len(train_index[cid]) // 10
        train_input_data, train_output_data = x_train[train_index[cid]], y_train[train_index[cid]]
        val_input_data, val_output_data = train_input_data[:val_size], train_output_data[:val_size]
        train_input_data, train_output_data = train_input_data[val_size:], train_output_data[val_size:]
        train_dataset = (train_input_data, train_output_data)
        val_dataset = (val_input_data, val_output_data)
        test_dataset = (x_test[test_index[cid]], y_test[test_index[cid]])
        train_ds.append(train_dataset)
        val_ds.append(val_dataset)
        test_ds.append(test_dataset)
    return train_ds, val_ds, test_ds


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Before continuing, it is a good Idea to make some test in order to check the correctness of our approach which could deal with this. For example, training a model on the first dataset

In [5]:
model = generate_ann()

model.fit(trainloaders[0][0],trainloaders[0][1], epochs=10, batch_size=64)
loss, accuracy = model.evaluate(valloaders[0][0], valloaders[0][1])
print(f"The resulting loss is {loss} for an accuracy in test of {accuracy*100:.4f}%")

Epoch 1/10


/home/ana/Documents/ml2-labs/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
2024-03-14 10:28:14.544947: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.1137 - loss: 2.2760
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1967 - loss: 2.1234
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2661 - loss: 1.9604
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3049 - loss: 1.8410
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3227 - loss: 1.7916
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3783 - loss: 1.7403
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4248 - loss: 1.5953
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4392 - loss: 1.5095
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4674 - loss: 1.5099
Epoch 10/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5303 - loss: 1.3545
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4176 - loss: 1.4736 
The resulting loss is 1.4906303882598877 for an accuracy in test of 43.0000%


**Important**: *The following example is thought to be executed in a terminal* 

Starting from that point, that it works on a particular data, the federated processs is going to split this process in several machines and for that it is going to require de definition of a couple of additional elements. Therefore, let's introduce the two pieces of the puzzle: the `Client` and the `Server`. 



Flower starts a `Server` to coordinate the client devices and perform the orchestration of the model. The server interacts with clients through an interface called `Client`. When the server selects a particular client for training, it sends training instructions over the network. The client receives those instructions and calls one of the Client methods to run your code, which in this case involves training the neural network that we defined earlier.

Flower provides a convenient class called NumPyClient, which simplifies the implementation of the Client interface when your workload uses Keras. The [NumPyClient](https://flower.ai/docs/framework/ref-api/flwr.client.NumPyClient.html#flwr.client.NumPyClient) interface defines three methods that can be implemented in the following way:

```python
#Create a class to contain the details of the client and be the interface
class MyClient(fl.client.NumPyClient):
    def __init__(self, net, train_dataset, test_dataset):
        self.model = net
        self.trainloader = train_dataset
        self.valloader = test_dataset
    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.trainloader[0],self.trainloader[1], epochs=1, batch_size=32, steps_per_epoch=3)
        return self.model.get_weights(), len(self.trainloader[0]), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.valloader[0], self.valloader[1])
        return loss, len(self.valloader[0]), {"accuracy": float(accuracy)}
```

In the preceding code, we defined the required functions for the client in this particular case. With these functions in place, we can now start a client using the following code:

```python 
# Start the client
model=generate_ann()

#The following line is deprecated in the current flower version
# fl.client.start_numpy_client(server_address="localhost:8080", client=MyClient(model,trainloaders[0],valloaders[0]))

fl.client.start_client(server_address=f"localhost:8080", client=MyClient(model,trainloaders[0],valloaders[0]).to_client())
```

**Important**: In order to run the client you must need also a server running!!! You will executed both in separated terminals

The string`localhost:8080` specifies the server to which the client should connect. In this case, as the code is being run on the same machine as the server, this address is sufficient. In a truly federated workload, the only thing that needs to be changed is the `server_address` to point the client to the correct server.

Note that Jupyter usually runs on port 8080, **so you will need to use another available port if Jupyter server is running**.

~~Possible exception: if you get the following error: "failed to connect to all addresses" the you shoul use the following string to stablish the connection `localhost:8080` instead of `[::]:8080`~~



The other essential piece of the puzzle is the class that will contain the server. This will be in a separate file, for example server.py, and its contents should look something like this:


```python
import flwr as fl

fl.server.start_server(config=fl.server.ServerConfig(num_rounds=3))
```

##### **Important**
You can use another port for running the server if you fix the following parameter in the `start_server` function: `server_address="localhost:8080"`

In this particular case, we can run two clients and a server in separate terminals of the machine. Running two client instances is as simple as executing the `python client.py` command twice in separate terminals, while the server can be started with the `python server.py` command.

Upon starting the server, we should receive an output similar to:


```shell
INFO flwr 2023-03-01 14:58:16,353 | app.py:139 | Starting Flower server, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flwr 2023-03-01 14:58:16,362 | app.py:152 | Flower ECE: gRPC server running (3 rounds), SSL is disabled
INFO flwr 2023-03-01 14:58:16,362 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-01 14:58:16,362 | server.py:270 | Requesting initial parameters from one random client
INFO flwr 2023-03-01 14:58:24,152 | server.py:274 | Received initial parameters from one random client
INFO flwr 2023-03-01 14:58:24,153 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-03-01 14:58:24,153 | server.py:101 | FL starting
DEBUG flwr 2023-03-01 14:58:26,118 | server.py:215 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:27,041 | server.py:229 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-03-01 14:58:27,076 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-03-01 14:58:27,076 | server.py:165 | evaluate_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:27,565 | server.py:179 | evaluate_round 1 received 2 results and 0 failures
WARNING flwr 2023-03-01 14:58:27,565 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-03-01 14:58:27,566 | server.py:215 | fit_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:28,015 | server.py:229 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-03-01 14:58:28,027 | server.py:165 | evaluate_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:28,364 | server.py:179 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2023-03-01 14:58:28,364 | server.py:215 | fit_round 3: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:28,755 | server.py:229 | fit_round 3 received 2 results and 0 failures
DEBUG flwr 2023-03-01 14:58:28,769 | server.py:165 | evaluate_round 3: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-03-01 14:58:29,184 | server.py:179 | evaluate_round 3 received 2 results and 0 failures
INFO flwr 2023-03-01 14:58:29,185 | server.py:144 | FL finished in 5.031599427999936
INFO flwr 2023-03-01 14:58:29,185 | app.py:202 | app_fit: losses_distributed [(1, 2.3956351280212402), (2, 2.426431179046631), (3, 2.3015435934066772)]
INFO flwr 2023-03-01 14:58:29,185 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-03-01 14:58:29,186 | app.py:204 | app_fit: losses_centralized []

```

With that, the first federated learning approach is completed. As you can see, the system goes through three rounds of fitting and evaluating on all clients before the results are retrieved, aggregated, and redistributed to the server.

### Exercise
Implement the client and server code in separate files (**All the required files must be submitted together with the Notebook**). Next, execute a server and two clients from terminals. Finally, compare the results with those presented here. Were your results similar?


**Note**:  All the required code to execute the server as well as the clients can be found this notebook. Considering the output displayed in the preceding cell, the following answer is expected to contain the corresponding terminal output resulting from the execution of your code.

`Answer here`:











```shell
INFO flwr 2024-03-12 14:20:02,531 | app.py:163 | Starting Flower server, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flwr 2024-03-12 14:20:02,554 | app.py:176 | Flower ECE: gRPC server running (3 rounds), SSL is disabled
INFO flwr 2024-03-12 14:20:02,554 | server.py:89 | Initializing global parameters
INFO flwr 2024-03-12 14:20:02,554 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2024-03-12 14:20:41,589 | server.py:280 | Received initial parameters from one random client
INFO flwr 2024-03-12 14:20:41,590 | server.py:91 | Evaluating initial parameters
INFO flwr 2024-03-12 14:20:41,590 | server.py:104 | FL starting
DEBUG flwr 2024-03-12 14:21:00,205 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-03-12 14:21:01,502 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2024-03-12 14:21:01,508 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-03-12 14:21:01,508 | server.py:173 | evaluate_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-03-12 14:21:02,098 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
WARNING flwr 2024-03-12 14:21:02,098 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2024-03-12 14:21:02,099 | server.py:222 | fit_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-03-12 14:21:02,543 | server.py:236 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2024-03-12 14:21:02,548 | server.py:173 | evaluate_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-03-12 14:21:03,076 | server.py:187 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2024-03-12 14:21:03,076 | server.py:222 | fit_round 3: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-03-12 14:21:03,539 | server.py:236 | fit_round 3 received 2 results and 0 failures
DEBUG flwr 2024-03-12 14:21:03,545 | server.py:173 | evaluate_round 3: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-03-12 14:21:03,967 | server.py:187 | evaluate_round 3 received 2 results and 0 failures
INFO flwr 2024-03-12 14:21:03,968 | server.py:153 | FL finished in 22.377772565003397
INFO flwr 2024-03-12 14:21:03,968 | app.py:226 | app_fit: losses_distributed [(1, 2.39928936958313), (2, 2.337975263595581), (3, 2.342345952987671)]
INFO flwr 2024-03-12 14:21:03,968 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-03-12 14:21:03,968 | app.py:228 | app_fit: metrics_distributed {}
INFO flwr 2024-03-12 14:21:03,968 | app.py:229 | app_fit: losses_centralized []
INFO flwr 2024-03-12 14:21:03,968 | app.py:230 | app_fit: metrics_centralized {}


```

# Updating parameters
The key element in this kind of approach is that the server sends the global model parameters to the client, and the client updates the local model with the parameters received from the server. It then trains the model on the local data, which changes the model parameters locally. After training, the updated model parameters are sent back to the server, or alternatively, only the gradients are sent back to the server, not the full model parameters.


In `flwr`, this communication is essentially done by two helper functions for loading and retrieving local parameters: `set_parameters` and `get_parameters`. This requirement fits well with non-state approaches such as **PyTorch** or **JAX**. As demonstrated in the previous example, `flwr` can also be used with **TensorFlow** or even **scikit-learn**.

As a result, the basic structure for any client using this library has the same format:


In [6]:
from typing import List

# Utility functions for the most common operations
def get_parameters(net) -> List[np.array]:
    return net.get_weights()


def set_parameters(net, parameters: List[np.ndarray]):
    net.set_weights(parameters)
    return net


def train(net, trainloader, epochs: int):
    net.fit(trainloader[0], trainloader[1],
            epochs=epochs, batch_size=32, steps_per_epoch=3)
    return net


def test(net, testloader):
    loss, accuracy = net.evaluate(testloader[0], testloader[1])
    return loss, accuracy

# Class to contain a Client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        self.net = set_parameters(self.net, parameters)
        self.net = train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        self.net = set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"[Client {self.cid}] loss:{loss}, Client {self.cid} accuracy:{accuracy}")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In Flower, clients can be created by extending either the `flwr.client.Client` or `flwr.client.NumPyClient` classes. In the previous example, we used `NumPyClient` because it is easier to implement and requires less code as a template. Along with the extended class, there are three main methods that need to be implemented:

* `get_parameters`: Returns the current local model parameters.
* `fit`: Receives model parameters from the server, trains the model parameters on the local data, and returns the (updated) model parameters to the server.
* `evaluate`: Receives model parameters from the server, evaluates the model parameters on the local data, and returns the evaluation result to the server.

As you can see, the `MyClient` class implemented in the previous example follows this same structure and the diference is the *id* of the client which is stored for later convinience use in accesing the data.


#### Be aware: 
Sometimes, especially when we are simulating multiple clients on a single device, it can be useful to use a function to create the client when it is required. This is particularly important in stateless frameworks, such as PyTorch, which can benefit from a more efficient implementation that creates clients only when they are required for training or evaluation. For example, the following code loads different examples for each client before discarding them:


In [7]:
def client_fn(cid) -> FlowerClient:
    # Create the model
    net = generate_ann()
    #Take the appropiate part of the dataset
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    #Create and return the Client
    return FlowerClient(cid, net, trainloader, valloader)


Note that `myClient` cannot be used in the same sense because of the state that it keeps internally through the function `generate_ann`. However, if this state is removed, it can be used in the same way.

The clients are now set up to load, fit, and evaluate. However, we need to integrate the results from the different clients. In Flower terminology, this is known as a strategy, such as the *Federated Average (FedAvg)* strategy. In a first approach, we can use the built-in implementations of the framework, although custom strategies can also be used. Let's see an example:


In [8]:
model = generate_ann()
params = get_parameters(model)# The federated model initial parameters
del model


# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # Sample 100% of available clients for training
        fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
        min_fit_clients=NUM_CLIENTS,  # Never sample less than NUM_CLIENTS clients for training
        min_evaluate_clients=NUM_CLIENTS//2,  # Never sample less than NUM_CLIENTS//2 clients for evaluation
        min_available_clients=NUM_CLIENTS,  # Wait until all NUM_CLIENTS clients are available
        initial_parameters=fl.common.ndarrays_to_parameters(params), # Initial parameters

)

# Start simulation. It allows to test on a single device. This code can be executed from the Notebook
fl.simulation.start_simulation(
    client_fn=client_fn, #function to create client instances
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,#strategy to integrate the model results
 )

INFO flwr 2024-03-14 10:28:18,688 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)


2024-03-14 10:28:20,363	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-14 10:28:21,247 | app.py:213 | Flower VCE: Ray initialized with resources: {'object_store_memory': 17456348774.0, 'CPU': 20.0, 'node:__internal_head__': 1.0, 'node:10.68.58.17': 1.0, 'accelerator_type:G': 1.0, 'GPU': 2.0, 'memory': 34912697550.0}
INFO flwr 2024-03-14 10:28:21,248 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-03-14 10:28:21,248 | app.py:227 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2024-03-14 10:28:21,248 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-03-14 10:28:21,260 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 20 actors
INFO flwr 2024-03-14 10:28:21,261 | server.py:89 | Initializing global parameters
INFO flwr 2024-03-14 10:28:21,261 | server.py:272 | Using ini

(DefaultActor pid=2921058) [Client 2] fit, config: {}


DEBUG flwr 2024-03-14 10:28:26,459 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2024-03-14 10:28:26,465 | fedavg.py:250 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-03-14 10:28:26,465 | server.py:173 | evaluate_round 1: strategy sampled 2 clients (out of 5)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.1081 - loss: 2.3061
(DefaultActor pid=2921059) [Client 1] evaluate, config: {}
(DefaultActor pid=2921059) [Client 1] loss:2.29203724861145, Client 1 accuracy:0.13500000536441803


...

DEBUG flwr 2024-03-14 10:28:37,570 | server.py:187 | evaluate_round 4 received 2 results and 0 failures
DEBUG flwr 2024-03-14 10:28:37,570 | server.py:222 | fit_round 5: strategy sampled 5 clients (out of 5)
(DefaultActor pid=2921055) WARNING:tensorflow:5 out of the last 13 calls to <function TensorFlowTrainer.make_train_function.<locals>.one_step_on_iterator at 0x7cf2b9fc1b40> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
(pid=2921059) 2024-03-14 10:28:23.423929: I tensorfl

(DefaultActor pid=2921059) [Client 2] fit, config: {} [repeated 6x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 421ms/step - accuracy: 0.1875 - loss: 2.1907 [repeated 2x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1569 - loss: 2.2188  


DEBUG flwr 2024-03-14 10:28:41,240 | server.py:187 | evaluate_round 5 received 2 results and 0 failures
INFO flwr 2024-03-14 10:28:41,241 | server.py:153 | FL finished in 19.97861633897992
INFO flwr 2024-03-14 10:28:41,244 | app.py:226 | app_fit: losses_distributed [(1, 2.293684959411621), (2, 2.2967350482940674), (3, 2.2843629121780396), (4, 2.2696338891983032), (5, 2.236458420753479)]
INFO flwr 2024-03-14 10:28:41,244 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-03-14 10:28:41,244 | app.py:228 | app_fit: metrics_distributed {}
INFO flwr 2024-03-14 10:28:41,245 | app.py:229 | app_fit: losses_centralized []
INFO flwr 2024-03-14 10:28:41,245 | app.py:230 | app_fit: metrics_centralized {}


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1457 - loss: 2.2316  


History (loss, distributed):
	round 1: 2.293684959411621
	round 2: 2.2967350482940674
	round 3: 2.2843629121780396
	round 4: 2.2696338891983032
	round 5: 2.236458420753479

This code corresponds to the script running on the server, and it uses the simulation function to test this approach on a single device with the previously mentioned optimization to avoid overloading the device. The code generates NUM_CLIENTS clients and randomly selects all of them (`fraction_fit = 1.0`) to train the model on all of them. After receiving the updates from the clients, the server performs the aggregation strategy before returning the global model to the clients for the next 5 rounds.

**Note**: *Be aware that the simulation is a really resource consuming task. You can get some errors and warnings linked to that. There are different [configurations](https://flower.ai/docs/framework/how-to-run-simulations.html) that you can apply to configure the available resources for simulating the process.*

One point to highlight is that the framework is not only going to manage the `losses_distributed`, but none of the other metrics. Due to the diverse treatment of those measures, the framework cannot accurately handle the aggregation of these metrics. Users need to tell the framework how to handle and aggregate these custom metrics.

The strategy will then call these functions whenever it receives fit or evaluates metrics from clients. The two possible functions are `fit_metrics_aggregation_fn` and `evaluate_metrics_aggregation_fn`. For example, the following code creates the weighted average, and the previous example can be adapted as follows:


In [9]:
from typing import Tuple, List
from flwr.common import Metrics

model = generate_ann()
params = get_parameters(model)# The federated model initial parameters
del model

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}


# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # Sample 100% of available clients for training
        fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
        min_fit_clients=NUM_CLIENTS,  # Never sample less than NUM_CLIENTS clients for training
        min_evaluate_clients=NUM_CLIENTS//2,  # Never sample less than NUM_CLIENTS//2 clients for evaluation
        min_available_clients=NUM_CLIENTS,  # Wait until all NUM_CLIENTS clients are available
        initial_parameters=fl.common.ndarrays_to_parameters(params), # Initial parameters
        evaluate_metrics_aggregation_fn=weighted_average,  # put the metric aggregation for the evaluation

)


# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
)


/home/ana/Documents/ml2-labs/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
INFO flwr 2024-03-14 10:28:41,288 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)


2024-03-14 10:28:44,862	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-14 10:28:45,775 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 20.0, 'node:__internal_head__': 1.0, 'memory': 34726839092.0, 'object_store_memory': 17363419545.0, 'node:10.68.58.17': 1.0, 'accelerator_type:G': 1.0, 'GPU': 2.0}
INFO flwr 2024-03-14 10:28:45,776 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-03-14 10:28:45,776 | app.py:227 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2024-03-14 10:28:45,777 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-03-14 10:28:45,789 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 20 actors
INFO flwr 2024-03-14 10:28:45,790 | server.py:89 | Initializing global parameters
INFO flwr 2024-03-14 10:28:45,790 | server.py:272 | Using ini

(DefaultActor pid=2923334) [Client 3] fit, config: {}


DEBUG flwr 2024-03-14 10:28:50,667 | server.py:236 | fit_round 1 received 5 results and 0 failures


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1680 - loss: 2.3026


...

DEBUG flwr 2024-03-14 10:29:01,702 | server.py:187 | evaluate_round 4 received 2 results and 0 failures
DEBUG flwr 2024-03-14 10:29:01,703 | server.py:222 | fit_round 5: strategy sampled 5 clients (out of 5)
(DefaultActor pid=2923334) WARNING:tensorflow:5 out of the last 13 calls to <function TensorFlowTrainer.make_train_function.<locals>.one_step_on_iterator at 0x774413fadb40> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
(pid=2923331) 2024-03-14 10:28:47.587256: I tensorfl

(DefaultActor pid=2923333) [Client 2] fit, config: {} [repeated 6x across cluster]


DEBUG flwr 2024-03-14 10:29:05,330 | server.py:187 | evaluate_round 5 received 2 results and 0 failures
INFO flwr 2024-03-14 10:29:05,331 | server.py:153 | FL finished in 19.539129890006734
INFO flwr 2024-03-14 10:29:05,333 | app.py:226 | app_fit: losses_distributed [(1, 2.297317385673523), (2, 2.297383189201355), (3, 2.295715570449829), (4, 2.284350037574768), (5, 2.274853229522705)]
INFO flwr 2024-03-14 10:29:05,333 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-03-14 10:29:05,334 | app.py:228 | app_fit: metrics_distributed {'accuracy': [(1, 0.11999999731779099), (2, 0.13749999925494194), (3, 0.11749999597668648), (4, 0.125), (5, 0.16250000149011612)]}
INFO flwr 2024-03-14 10:29:05,334 | app.py:229 | app_fit: losses_centralized []
INFO flwr 2024-03-14 10:29:05,334 | app.py:230 | app_fit: metrics_centralized {}


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 418ms/step - accuracy: 0.2500 - loss: 2.2565


History (loss, distributed):
	round 1: 2.297317385673523
	round 2: 2.297383189201355
	round 3: 2.295715570449829
	round 4: 2.284350037574768
	round 5: 2.274853229522705
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.11999999731779099), (2, 0.13749999925494194), (3, 0.11749999597668648), (4, 0.125), (5, 0.16250000149011612)]}

We will revisit the definition of custom strategies in the following unit to define our own strategy and attempt to minimize some of the challenges that federated learning must address.

### Exercise

Now is your turn, why not you try to run your own architecture with this approach. Beaware of the high requirements when we are in a simulated environment.

**Note**: The objective of this exercise is to bring together the various topics covered by this notebook. This cell should contain all the necessary code for autonomous execution using Flower simulation.

In [10]:
import tensorflow as tf

# Code to load the dataset
import numpy as np

NUM_CLIENTS = 4

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def split_index(a, n):
    s = np.array_split(np.arange(len(a)), n)
    return s


def generate_ann():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


# Code to load the dataset
def load_datasets(num_clients: int):
    # Distribute it to train and test set
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    # Normalize data
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    x_train, y_train = x_train[:10_000], y_train[:10_000]
    x_test, y_test = x_test[:1000], y_test[:1000]

    # Randomize the datasets
    x_train, y_train = unison_shuffled_copies(x_train, y_train)
    x_test, y_test = unison_shuffled_copies(x_test, y_test)

    # Split training set into num_clients partitions to simulate the individual dataset
    train_index = split_index(x_train, num_clients)
    test_index = split_index(x_test, num_clients)

    # Split each partition
    train_ds = []
    val_ds = []
    test_ds = []
    for cid in range(num_clients):
        val_size = len(train_index[cid]) // 10
        train_input_data, train_output_data = x_train[train_index[cid]], y_train[train_index[cid]]
        val_input_data, val_output_data = train_input_data[:val_size], train_output_data[:val_size]
        train_input_data, train_output_data = train_input_data[val_size:], train_output_data[val_size:]
        train_dataset = (train_input_data, train_output_data)
        val_dataset = (val_input_data, val_output_data)
        test_dataset = (x_test[test_index[cid]], y_test[test_index[cid]])
        train_ds.append(train_dataset)
        val_ds.append(val_dataset)
        test_ds.append(test_dataset)
    return train_ds, val_ds, test_ds


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

#TODO Client, client_fn and simulation
# Utility functions for the most common operations
def get_parameters(net) -> List[np.array]:
    return net.get_weights()


def set_parameters(net, parameters: List[np.ndarray]):
    net.set_weights(parameters)
    return net


def train(net, trainloader, epochs: int):
    net.fit(trainloader[0], trainloader[1],
            epochs=epochs, batch_size=32)#, steps_per_epoch=3)
    return net


def test(net, testloader):
    loss, accuracy = net.evaluate(testloader[0], testloader[1])
    return loss, accuracy

# Class to contain a Client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        self.net = set_parameters(self.net, parameters)
        self.net = train(self.net, self.trainloader, epochs=10)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        self.net = set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"[Client {self.cid}] loss:{loss}, Client {self.cid} accuracy:{accuracy}")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
    # Create the model
    net = generate_ann()
    #Take the appropiate part of the dataset
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    #Create and return the Client
    return FlowerClient(cid, net, trainloader, valloader)

#Simulation code
model = generate_ann()
params = get_parameters(model)# The federated model initial parameters
del model

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}


# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # Sample 100% of available clients for training
        fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
        min_fit_clients=NUM_CLIENTS,  # Never sample less than NUM_CLIENTS clients for training
        min_evaluate_clients=NUM_CLIENTS//2,  # Never sample less than NUM_CLIENTS//2 clients for evaluation
        min_available_clients=NUM_CLIENTS,  # Wait until all NUM_CLIENTS clients are available
        initial_parameters=fl.common.ndarrays_to_parameters(params), # Initial parameters
        evaluate_metrics_aggregation_fn=weighted_average,  # put the metric aggregation for the evaluation
)


# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2048 - loss: 2.2669  
(DefaultActor pid=2923335) [Client 1] loss:2.269329309463501, Client 1 accuracy:0.1850000023841858


/home/ana/Documents/ml2-labs/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
INFO flwr 2024-03-14 10:29:06,217 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2024-03-14 10:29:09,558	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-14 10:29:10,467 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 20.0, 'node:__internal_head__': 1.0, 'node:10.68.58.17': 1.0, 'memory': 34592835995.0, 'GPU': 2.0, 'accelerator_type:G': 1.0, 'object_store_memory': 17296417996.0}
INFO flwr 2024-03-14 10:29:10,467 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-03-14 10:29:10,468 | app.py:227 | No `clie

(DefaultActor pid=2925611) [Client 2] fit, config: {}
(DefaultActor pid=2925611) Epoch 1/10
 6/71 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1000 - loss: 2.3062
18/71 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1209 - loss: 2.2997
19/71 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1217 - loss: 2.2967
37/71 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1245 - loss: 2.2884
 1/71 ━━━━━━━━━━━━━━━━━━━━ 1:32 1s/step - accuracy: 0.1250 - loss: 2.2974
56/71 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1280 - loss: 2.2784
10/71 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1183 - loss: 2.3018
20/71 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1163 - loss: 2.2950
 5/71 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2510 - loss: 2.0262
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3420 - loss: 1.7717
(DefaultActor pid=2925612) [Client 0] fit, config: {} [repeated 3x across cluster]
(DefaultActor pid=2925611) Epoch 5/10 [repeated 17x across cluster]
 5/71 ━━━━━━━━━━━━━━━

DEBUG flwr 2024-03-14 10:29:24,796 | server.py:236 | fit_round 1 received 4 results and 0 failures
WARNING flwr 2024-03-14 10:29:24,801 | fedavg.py:250 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-03-14 10:29:24,801 | server.py:173 | evaluate_round 1: strategy sampled 2 clients (out of 4)


71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6571 - loss: 0.9662
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6036 - loss: 1.1144
(DefaultActor pid=2925612) [Client 1] evaluate, config: {}
(DefaultActor pid=2925612) [Client 1] loss:1.9293136596679688, Client 1 accuracy:0.42399999499320984
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4302 - loss: 1.9130   [repeated 11x across cluster]


...

DEBUG flwr 2024-03-14 10:29:49,343 | server.py:236 | fit_round 3 received 4 results and 0 failures
DEBUG flwr 2024-03-14 10:29:49,347 | server.py:173 | evaluate_round 3: strategy sampled 2 clients (out of 4)


(DefaultActor pid=2925613) [Client 0] evaluate, config: {}
(DefaultActor pid=2925613) Epoch 10/10 [repeated 18x across cluster]
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9889 - loss: 0.0425 [repeated 3x across cluster]
(DefaultActor pid=2925613) [Client 0] loss:2.496349573135376, Client 0 accuracy:0.47600001096725464


DEBUG flwr 2024-03-14 10:29:50,159 | server.py:187 | evaluate_round 3 received 2 results and 0 failures
INFO flwr 2024-03-14 10:29:50,160 | server.py:153 | FL finished in 39.67344760199194
INFO flwr 2024-03-14 10:29:50,160 | app.py:226 | app_fit: losses_distributed [(1, 1.9540851712226868), (2, 1.6804097294807434), (3, 2.3646175861358643)]
INFO flwr 2024-03-14 10:29:50,160 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-03-14 10:29:50,161 | app.py:228 | app_fit: metrics_distributed {'accuracy': [(1, 0.40799999237060547), (2, 0.5060000121593475), (3, 0.5180000066757202)]}
INFO flwr 2024-03-14 10:29:50,161 | app.py:229 | app_fit: losses_centralized []
INFO flwr 2024-03-14 10:29:50,161 | app.py:230 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 1.9540851712226868
	round 2: 1.6804097294807434
	round 3: 2.3646175861358643
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.40799999237060547), (2, 0.5060000121593475), (3, 0.5180000066757202)]}

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5279 - loss: 2.3875  
(DefaultActor pid=2925612) [Client 2] loss:2.2328855991363525, Client 2 accuracy:0.5600000023841858


# Aggregation

To conclude this lesson, let's take a closer look at the key point of these strategies, which is the aggregation algorithm. These algorithms are responsible for combining the updates from the clients to generate the global model, and they are defined in the strategies as we have seen. Generally speaking, there are several types of aggregation that can be used in federated learning (Reddi et. al, 2020).  

Here are the different types of aggregation that can be used in federated learning:

* Federated averaging (`flwr.server.strategy.FedAvg`): In this approach, each device computes an update to the model parameters based on its local data, and these updates are then averaged together to create the global model. This approach is simple and effective, but it can be sensitive to the size of the updates and the quality of the data on each device.

* Federated weighted averaging: This approach is similar to federated averaging, but each device's update is given a different weight based on the size of its data set or the quality of its data. This can help to give more influence to devices with larger or higher-quality data.

* Federated averaging with momentum (`flwr.server.strategy.FedAvgM`): This approach is similar to federated averaging, but it incorporates a momentum term in order to smooth out the updates and help the model converge more quickly.

* Federated stochastic gradient descent(`flwr.server.strategy.FedAdagrad`): In this approach, each device computes an update to the model parameters based on a small batch of its local data, rather than the entire data set. This can help to reduce the communication overhead and improve the convergence rate of the model.

* Federated ADAM (`flwr.server.strategy.FedAdam`): This approach is a variant of federated stochastic gradient descent that uses the ADAM optimization algorithm to adaptively adjust the learning rate based on the gradient and second moment estimates.



All of the previously mentioned aggregation methods, except for Federated Weighted Averaging, are implemented in the `flwr` framework and can be used with the different strategies. Additionally, there are other less common aggregation methods that can be employed. The choice of aggregation method will ultimately depend on the specific characteristics of the data and the requirements of the task at hand.


#### References
* Hard, A., Konečný, J., McMahan, H. B., Richemond-Barakat, C., Sivek, J. S., & Talwar, K. (2018). Federated learning: Strategies for improving communication efficiency. arXiv preprint arXiv:1812.02903.
* Li, Y., Bonawitz, K., & Talwar, K. (2020). Fedprox: An optimizer for communication-efficient federated learning. arXiv preprint arXiv:2002.04283.
* McMahan, H. B., Moore, E., Ramage, D., Hampson, S., & y Arcas, B. A. (2016). Communication-efficient learning of deep networks from decentralized data. arXiv preprint arXiv:1602.05629.
* Yoon, J., Hard, A., Konečný, J., McMahan, H. B., & Sohl-Dickstein, J. (2018). Federal regression: A simple and scalable method for heterogeneous federated learning. arXiv preprint arXiv:1812.03862.
* Reddi, S., Charles, Z., Zaheer, M., Garrett, Z., Rush, K., Konečný, J., Kumar, S. and McMahan, H.B., 2020. Adaptive federated optimization. arXiv preprint arXiv:2003.00295.